In [0]:
from google.colab import drive

In [32]:
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/EIP3Phase2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
model	saved_models	 Session1_1.ipynb  wonderland.txt
model2	session1_1.hdf5  Session1_2.ipynb


In [0]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/EIP3Phase2/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


In [34]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
print("len = ", len(chars), chars)
puncts = [',', '!', '#', '$', '%', '*', ':', '@', ';', '?', '-', '/', '_', ]
print("len = ", len(puncts), puncts)

sourceset = list(raw_text)
for x in puncts:
   [sourceset.remove(x) for Item in range(sourceset.count(x))]

#chars = [x for x in puncts]
[chars.remove(item) for item in puncts]
print("len = ", len(chars), chars)

len =  59 ['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
len =  13 [',', '!', '#', '$', '%', '*', ':', '@', ';', '?', '-', '/', '_']
len =  46 ['\n', ' ', '"', "'", '(', ')', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [35]:

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_srcchars = len(sourceset)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Source Characters: ", n_srcchars)
print("Total Vocab: ", n_vocab)

raw_text = sourceset
n_chars = n_srcchars

Total Characters:  163781
Total Source Characters:  159247
Total Vocab:  46


In [66]:
from keras.preprocessing.sequence import pad_sequences

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
  
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# padding doesn't make sense for this as seq_length is fixed here
# However for completeness of assignment using this here with 1 extra chars
savedDataX = dataX
dataX = pad_sequences(dataX, maxlen=101)
n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, 101, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.2))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Total Patterns:  159147
Total Patterns:  159147


In [41]:
filepath="/content/drive/My Drive/EIP3Phase2/model2/weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
159147/159147 [==============================] - 336s 2ms/step - loss: 2.9071

Epoch 00001: loss improved from inf to 2.90714, saving model to /content/drive/My Drive/EIP3Phase2/model2/weights-01-2.9071.hdf5
Epoch 2/20
159147/159147 [==============================] - 331s 2ms/step - loss: 2.7943

Epoch 00002: loss improved from 2.90714 to 2.79428, saving model to /content/drive/My Drive/EIP3Phase2/model2/weights-02-2.7943.hdf5
Epoch 3/20
159147/159147 [==============================] - 332s 2ms/step - loss: 2.6952

Epoch 00003: loss improved from 2.79428 to 2.69523, saving model to /content/drive/My Drive/EIP3Phase2/model2/weights-03-2.6952.hdf5
Epoch 4/20
159147/159147 [==============================] - 332s 2ms/step - loss: 2.6025

Epoch 00004: loss improved from 2.69523 to 2.60251, saving model to /content/drive/My Drive/EIP3Phase2/model2/weights-04-2.6025.hdf5
Epoch 5/20
159147/159147 [==============================] - 334s 2ms/step - loss: 2.5316

Epoch 00005: loss impr

In [63]:
# load the network weights
from keras.models import load_model

filepath2="/content/drive/My Drive/EIP3Phase2/model2/weights-20-2.0081.hdf5"
model = load_model('/content/drive/My Drive/EIP3Phase2/model2/weights-20-2.0081.hdf5')
#model.load_weights(filepath2)
#model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
print(len(dataX))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed: Len", len(pattern))
#print(pattern)

print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters


159147
Seed: Len 101
" 
n waiting by the little door so she went
back to the table half hoping she might find another key on "


In [64]:

for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  #print(index)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern = numpy.append(pattern, index)
  pattern = pattern[1:len(pattern)]
  
print("\nDone.")

nh tonet oolhr onad ienns a didielelelelenen a didnnsyenes toketeyk toketeyk toketeyeleie ioat tonet oel brl tokete ioennser ons tonet oadl ons toketenen soketenedne nonh toketenennsy toketenedne nonhi ioenes toketenedne nonhi ioenes toketenedne nonh toketenennsenei b dicens to an enl nosety a dicens a. 
a.1

11.  ioicnsenel ot a0  a0  ioaies a de71  ioent to tiatl tonet oolh toketeyeleie i tonet onad ioannsyenes toketeyk toketeyk toketeyk toketeyk toketeyeleie ioat tonet oel brl toketenedne ton
Done.


In [69]:
# An earlier model without the padded sequences has better pridictions
# load the network weights

filename = "/content/drive/My Drive/EIP3Phase2/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

model = load_model('/content/drive/My Drive/EIP3Phase2/model/weights-20-1.6101.hdf5')
#model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  163781
Total Vocab:  59
Total Patterns:  163681
Seed:
" took me for his housemaid,' she said to herself as she ran. 'how
surprised he'll be when he finds ou "
t of the wouds beain, and the doom thing that she was not a little boorer of the sabbit sas the danled of the sabbit sas the danled oo the thoee of the garden with the tooke. 
'it was a mine things,' said the ming, 'it would be a little gord the door tiat she was a little bool the little boos in the thme, and the was talking about the little gord of the tooe of the words as she could her hn the tooke. 
'it was a reree of the soiee,                                                                 
Done.
